In [162]:
import os
import boto3
import re
from sagemaker import get_execution_role
#retrives the IAM role created at the time of creating the notebook instance
role = get_execution_role()
bucket='ccprojectmlpart'
prefix = 'sagemaker/cc-ml-part' # place to upload training files within the bucket

In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

In [164]:
#data = pd.read_csv('https://raw.githubusercontent.com/CCLocationSharing/Real-time-Location-Sharing-System/data/mldata/data.csv', header = None)
data = pd.read_csv('https://raw.githubusercontent.com/CCLocationSharing/Real-time-Location-Sharing-System/data/mldata/ccMlData.csv', header = None)
# specify columns extracted from wbdc.names

#data.columns = ["libID","Quiet","Monitor", "carpenter", "olin", "uris", "gates"] 
#data.columns = ["id","default","student", "balance", "income"] 
data.columns = ["result","varone", "vartwo", "varthree", "varfour"] 

# save the data
data.to_csv("data.csv", sep=',', index=False)

# print the shape of the data file
print(data.shape)

# show the top few rows
display(data.head())

# describe the data object
display(data.describe())

# we will also summarize the categorical field diganosis 
#display(data.libID.value_counts())
#display(data.result.value_counts())


(50, 5)


,result,varone,vartwo,varthree,varfour
0,NO,0.519938,0.173180,0.633713,0.202430
1,NO,0.136409,0.481856,0.424786,0.649426
2,NO,0.023278,0.394771,0.562469,0.311281
3,NO,0.756794,0.014019,0.336345,0.275777
4,NO,0.492956,0.074432,0.552733,0.644069


,varone,vartwo,varthree,varfour
count,50.000000,50.000000,50.000000,50.000000
mean,0.475385,0.527252,0.572910,0.490557
std,0.270341,0.290081,0.270727,0.277395
min,0.010261,0.014019,0.032338,0.050878
25%,0.292841,0.315270,0.359292,0.231376
50%,0.480992,0.566433,0.586123,0.531381
75%,0.698647,0.753215,0.779030,0.649322
max,0.943509,0.964364,0.987397,0.988831


In [165]:
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

data_train = data[train_list]
data_val = data[val_list]
data_test = data[test_list]

'''train_y = data_train.iloc[:,0].as_matrix();
train_X = data_train.iloc[:,1:].as_matrix();

val_y = data_val.iloc[:,0].as_matrix();
val_X = data_val.iloc[:,1:].as_matrix();

test_y = data_test.iloc[:,0].as_matrix();
test_X = data_test.iloc[:,1:].as_matrix();
'''
train_y = ((data_train.iloc[:,0] == 'YES') +0).as_matrix();
train_X = data_train.iloc[:,1:].as_matrix();

val_y = ((data_val.iloc[:,0] == 'YES') +0).as_matrix();
val_X = data_val.iloc[:,1:].as_matrix();

test_y = ((data_test.iloc[:,0] == 'YES') +0).as_matrix();
test_X = data_test.iloc[:,1:].as_matrix();


In [166]:
print(train_y)
print(train_X)

[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 1 0 0
 1 0 1 0 1]
[[0.51993766 0.17318025 0.63371295 0.20242995]
 [0.13640929 0.48185619 0.42478645 0.64942554]
 [0.02327758 0.39477098 0.56246897 0.31128063]
 [0.7567938  0.01401877 0.3363448  0.27577676]
 [0.02512522 0.26670951 0.59530745 0.16573655]
 [0.78154871 0.11528718 0.95544239 0.91645942]
 [0.9435091  0.61716638 0.93558834 0.21657582]
 [0.73313068 0.52364606 0.98739705 0.59304095]
 [0.60682575 0.0781276  0.39936557 0.96186042]
 [0.83456227 0.92460014 0.54947621 0.83661171]
 [0.48111848 0.07000166 0.60175019 0.64901302]
 [0.80189377 0.89769405 0.68219139 0.35680329]
 [0.44058437 0.31114204 0.90655993 0.57597697]
 [0.69667095 0.40909037 0.76919531 0.54789997]
 [0.09703876 0.9236951  0.57693808 0.83189458]
 [0.19985    0.73498447 0.03535947 0.8534839 ]
 [0.51452358 0.15549792 0.53456131 0.53142992]
 [0.56592684 0.55773727 0.74315405 0.34594877]
 [0.05513692 0.38505634 0.40651158 0.49125867]
 [0.60177593 0.45387

In [167]:
#convert the datasets to the recordIO-wrapped protobuf format used by the Amazon SageMaker algorithms, and then upload this data to S3.
train_file = 'linear_train.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype('float32'), train_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', train_file)).upload_fileobj(f)

In [168]:
#convert and upload the validation dataset.
validation_file = 'linear_validation.data'

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype('float32'), val_y.astype('float32'))
f.seek(0)

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', validation_file)).upload_fileobj(f)

In [169]:
# See 'Algorithms Provided by Amazon SageMaker: Common Parameters' in the SageMaker documentation for an explanation of these values.
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}

In [170]:
#Train

linear_job = 'cc-ml-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())



print("Job name is:", linear_job)

linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {
        "TrainingImage": containers[boto3.Session().region_name],
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.c4.2xlarge",
        "VolumeSizeInGB": 10
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }

    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/".format(bucket, prefix)
    },
    "HyperParameters": {
        "feature_dim": "4",
        "mini_batch_size": "100",
        "predictor_type": "regressor",
        "epochs": "10",
        "num_models": "auto",
        "loss": "auto"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

Job name is: cc-ml-2018-05-01-21-33-11


In [171]:
%%time

region = boto3.Session().region_name
sm = boto3.client('sagemaker')

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)['TrainingJobStatus']
print(status)
sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=linear_job)
if status == 'Failed':
    message = sm.describe_training_job(TrainingJobName=linear_job)['FailureReason']
    print('Training failed with the following error: {}'.format(message))
    raise Exception('Training job failed')

InProgress


WaiterError: Waiter TrainingJobCompletedOrStopped failed: Waiter encountered a terminal failure state

In [176]:
print(sm.waiter_names)

['endpoint_deleted', 'endpoint_in_service', 'notebook_instance_deleted', 'notebook_instance_in_service', 'notebook_instance_stopped', 'training_job_completed_or_stopped']


In [173]:
print(sm)

In [175]:
print(sm.class())

SyntaxError: invalid syntax (<ipython-input-175-cb07f82168d7>, line 1)